**[中級機械学習ホームページ](https://www.kaggle.com/learn/intermediate-machine-learning)**

---


**カテゴリ変数**をエンコードすることで、これまでで最良の結果が得られる！

# セットアップ

以下の質問は作業に対するフィードバックを提供する。フィードバックシステムを設定するために次のセルを実行しよう。

In [1]:
# Kaggle学習環境のコード確認設定
# from learntools.core import binder
# binder.bind(globals())
# from learntools.ml_intermediate.ex3 import *
# print("Setup Complete")

このエクササイズでは、[Kaggle学習ユーザー向け住宅価格コンペティション](https://www.kaggle.com/c/home-data-for-ml-course)のデータを使用する。

![Ames住宅データセット画像](https://i.imgur.com/lTJVG4e.png)

次のコードセルを変更せずに実行して、訓練セットと検証セットを`X_train`、`X_valid`、`y_train`、`y_valid`に読み込む。テストセットは`X_test`に読み込まれる。

In [2]:
# --- データファイルの展開と確認 ---
#   データディレクトリ内のファイル一覧を表示する。
!ls ./input/Housing-Prices 
!echo "---------------"
# !gzip -d -c ./input/Housing-Prices/train.csv.gz > ./input/Housing-Prices/train.csv
#   train.csv.gz（圧縮ファイル）を解凍してtrain.csvを作成する。
!gzip -d -c ./input/Housing-Prices/train.csv.gz > ./input/Housing-Prices/train.csv
# !gzip -d -c ./input/Housing-Prices/test.csv.gz > ./input/Housing-Prices/test.csv
#   test.csv.gz（圧縮ファイル）を解凍してtest.csvを作成する。
!gzip -d -c ./input/Housing-Prices/test.csv.gz > ./input/Housing-Prices/test.csv
# !ls ./input/Housing-Prices
#   再度ファイル一覧を表示し、解凍後のファイルが存在するか確認する。
!ls ./input/Housing-Prices 


test.csv  test.csv.gz  train.csv  train.csv.gz
---------------
test.csv  test.csv.gz  train.csv  train.csv.gz


In [2]:
import pandas as pd
from sklearn.model_selection import train_test_split

# データの読み込み
# index_col='Id'でIDを行インデックスとして設定
X = pd.read_csv('../input/train.csv', index_col='Id') 
X_test = pd.read_csv('../input/test.csv', index_col='Id')

# 目的変数が欠損している行を削除し、目的変数と説明変数を分離
# dropna: 欠損値を含む行を削除
# axis=0: 行方向の削除
# subset=['SalePrice']: SalePriceが欠損している行のみを対象
# inplace=True: 元のデータフレームを変更
X.dropna(axis=0, subset=['SalePrice'], inplace=True)
# 目的変数（住宅価格）を取り出す
y = X.SalePrice
# 説明変数から目的変数の列を削除
X.drop(['SalePrice'], axis=1, inplace=True)

# シンプルにするため、欠損値を含む列を削除
# リスト内包表記を使用: X.columns内の各列colに対して、X[col].isnull().any()がTrueなら列名を取得
cols_with_missing = [col for col in X.columns if X[col].isnull().any()] 
X.drop(cols_with_missing, axis=1, inplace=True)
X_test.drop(cols_with_missing, axis=1, inplace=True)

# 訓練データから検証データを分割
# train_size=0.8: 訓練データに80%を使用
# test_size=0.2: 検証データに20%を使用
# random_state=0: 再現性のための乱数シード
X_train, X_valid, y_train, y_valid = train_test_split(X, y,
                                                      train_size=0.8, test_size=0.2,
                                                      random_state=0)

次のコードセルを使用して、データの最初の5行を表示する。

In [3]:
# head()メソッドでデータフレームの先頭5行を表示
X_train.head()

,MSSubClass,MSZoning,LotArea,Street,LotShape,LandContour,Utilities,LotConfig,LandSlope,Neighborhood,...,OpenPorchSF,EnclosedPorch,3SsnPorch,ScreenPorch,PoolArea,MiscVal,MoSold,YrSold,SaleType,SaleCondition
Id,,,,,,,,,,,,,,,,,,,,,
619,20,RL,11694,Pave,Reg,Lvl,AllPub,Inside,Gtl,NridgHt,...,108,0,0,260,0,0,7,2007,New,Partial
871,20,RL,6600,Pave,Reg,Lvl,AllPub,Inside,Gtl,NAmes,...,0,0,0,0,0,0,8,2009,WD,Normal
93,30,RL,13360,Pave,IR1,HLS,AllPub,Inside,Gtl,Crawfor,...,0,44,0,0,0,0,8,2009,WD,Normal
818,20,RL,13265,Pave,IR1,Lvl,AllPub,CulDSac,Gtl,Mitchel,...,59,0,0,0,0,0,7,2008,WD,Normal
303,20,RL,13704,Pave,IR1,Lvl,AllPub,Corner,Gtl,CollgCr,...,81,0,0,0,0,0,1,2006,WD,Normal


このデータセットには数値変数とカテゴリ変数の両方が含まれていることに注目。モデルを訓練する前にカテゴリデータをエンコードする必要がある。

異なるモデルを比較するために、チュートリアルと同じ`score_dataset()`関数を使用する。この関数はランダムフォレストモデルから[平均絶対誤差](https://en.wikipedia.org/wiki/Mean_absolute_error) (MAE)を報告する。

In [4]:
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_absolute_error

# 異なるアプローチを比較するための関数
def score_dataset(X_train, X_valid, y_train, y_valid):
    # ランダムフォレスト回帰モデルを初期化（100本の決定木、乱数シード0）
    model = RandomForestRegressor(n_estimators=100, random_state=0)
    # モデルを訓練データにフィット（学習）させる
    model.fit(X_train, y_train)
    # 検証データに対する予測を生成
    preds = model.predict(X_valid)
    # 予測値と実際の値の平均絶対誤差を計算して返す
    return mean_absolute_error(y_valid, preds)

# ステップ1: カテゴリデータを含む列を削除する

最も単純なアプローチから始める。以下のコードセルを使用して、`X_train`と`X_valid`のデータを前処理し、カテゴリデータを含む列を削除する。前処理されたDataFrameをそれぞれ`drop_X_train`と`drop_X_valid`に設定する。

In [5]:
# 訓練データと検証データからカテゴリ列を削除
# select_dtypes(exclude=['object'])で文字列型（オブジェクト型）の列を除外
drop_X_train = X_train.select_dtypes(exclude=['object'])
drop_X_valid = X_valid.select_dtypes(exclude=['object'])

# Kaggle環境での回答確認用コード（コメントアウト）
# step_1.check()

<IPython.core.display.Javascript object>

<span style="color:#33cc33">Correct</span>

In [6]:
# ヒントや解答コードを表示するための行（Kaggle環境用）
#step_1.hint()
#step_1.solution()

このアプローチのMAEを取得するために次のコードセルを実行する。

In [7]:
print("MAE from Approach 1 (Drop categorical variables):")
print(score_dataset(drop_X_train, drop_X_valid, y_train, y_valid))

MAE from Approach 1 (Drop categorical variables):
17837.82570776256


# ステップ2: ラベルエンコーディング

ラベルエンコーディングに進む前に、データセットを調査する。具体的には、`'Condition2'`列を見てみる。以下のコードセルは、訓練セットと検証セットの両方でのユニークな値を表示する。

In [8]:
# unique()メソッドで列内のユニークな値を取得
print("Unique values in 'Condition2' column in training data:", X_train['Condition2'].unique())
print("\nUnique values in 'Condition2' column in validation data:", X_valid['Condition2'].unique())

Unique values in 'Condition2' column in training data: ['Norm' 'PosA' 'Feedr' 'PosN' 'Artery' 'RRAe']

Unique values in 'Condition2' column in validation data: ['Norm' 'RRAn' 'RRNn' 'Artery' 'Feedr' 'PosN']


ここで、以下のコードを書いた場合：
- 訓練データにラベルエンコーダーをフィットし、
- それを使って訓練データと検証データの両方を変換する

エラーが発生する。なぜこれが問題になるか分かるだろうか？（この質問に答えるには、上記の出力を使用する必要がある）

In [9]:
# Kaggle環境でのヒント表示用（コメントアウト）
# step_2.a.hint()

In [10]:
# Kaggle環境での解答表示用（コメントアウト）
# step_2.a.solution()

<IPython.core.display.Javascript object>

<span style="color:#33cc99">Solution:</span> Fitting a label encoder to a column in the training data creates a corresponding integer-valued label for each unique value **that appears in the training data**. In the case that the validation data contains values that don't also appear in the training data, the encoder will throw an error, because these values won't have an integer assigned to them.  Notice that the `'Condition2'` column in the validation data contains the values `'RRAn'` and `'RRNn'`, but these don't appear in the training data -- thus, if we try to use a label encoder with scikit-learn, the code will throw an error.

これは実世界のデータでよく遭遇する一般的な問題であり、この問題を解決するためのアプローチは多数ある。例えば、新しいカテゴリに対応するカスタムラベルエンコーダーを作成することができる。しかし、最も単純なアプローチは、問題のあるカテゴリ列を削除することだ。

以下のコードセルを実行して、問題のある列をPythonリスト`bad_label_cols`に保存する。同様に、安全にラベルエンコードできる列は`good_label_cols`に格納される。

In [11]:
# すべてのカテゴリ列
# dtype == "object"でカテゴリ（文字列）型の列を特定
object_cols = [col for col in X_train.columns if X_train[col].dtype == "object"]

# 安全にラベルエンコードできる列
# 訓練データと検証データで同じ値のセットを持つ列
good_label_cols = [col for col in object_cols if 
                   set(X_train[col]) == set(X_valid[col])]
        
# データセットから削除される問題のある列
# 集合の差集合演算で、すべてのカテゴリ列から安全な列を引く
bad_label_cols = list(set(object_cols)-set(good_label_cols))
        
print('Categorical columns that will be label encoded:', good_label_cols)
print('\nCategorical columns that will be dropped from the dataset:', bad_label_cols)

Categorical columns that will be label encoded: ['MSZoning', 'Street', 'LotShape', 'LandContour', 'LotConfig', 'BldgType', 'HouseStyle', 'ExterQual', 'CentralAir', 'KitchenQual', 'PavedDrive', 'SaleCondition']

Categorical columns that will be dropped from the dataset: ['Neighborhood', 'RoofStyle', 'ExterCond', 'Functional', 'Condition2', 'Heating', 'Exterior2nd', 'RoofMatl', 'Utilities', 'Foundation', 'LandSlope', 'HeatingQC', 'Condition1', 'SaleType', 'Exterior1st']


以下のコードセルを使用して、`X_train`と`X_valid`のデータをラベルエンコードする。前処理されたDataFrameをそれぞれ`label_X_train`と`label_X_valid`に設定する。

- `bad_label_cols`のカテゴリ列をデータセットから削除するコードを提供している。
- `good_label_cols`のカテゴリ列をラベルエンコードする必要がある。

In [12]:
from sklearn.preprocessing import LabelEncoder

# エンコードされないカテゴリ列を削除
label_X_train = X_train.drop(bad_label_cols, axis=1)
label_X_valid = X_valid.drop(bad_label_cols, axis=1)

# ラベルエンコーダーを適用
labelEncoder = LabelEncoder()
for col in good_label_cols:
    # 訓練データでエンコーダーをフィットし、変換
    label_X_train[col] = labelEncoder.fit_transform(X_train[col])
    # 検証データを変換（フィットはしない）
    label_X_valid[col] = labelEncoder.transform(X_valid[col])
    
# Kaggle環境での回答確認用（コメントアウト）
# step_2.b.check()

<IPython.core.display.Javascript object>

<span style="color:#33cc33">Correct</span>

In [13]:
# ヒントや解答コードを表示するための行（Kaggle環境用）
#step_2.b.hint()
#step_2.b.solution()

このアプローチのMAEを取得するために次のコードセルを実行する。

In [14]:
print("MAE from Approach 2 (Label Encoding):") 
print(score_dataset(label_X_train, label_X_valid, y_train, y_valid))

MAE from Approach 2 (Label Encoding):
17575.291883561644


# ステップ3: カーディナリティの調査

これまで、カテゴリ変数を扱うための2つの異なるアプローチを試した。そして、カテゴリデータをエンコードすることが、データセットから列を削除するよりも良い結果をもたらすことがわかった。

次に、ワンホットエンコーディングを試す。その前に、もう一つ追加のトピックをカバーする必要がある。まず、次のコードセルを変更せずに実行する。

In [15]:
# カテゴリデータを持つ各列のユニークな値の数を取得
# map関数で各列に対してnunique()を適用
object_nunique = list(map(lambda col: X_train[col].nunique(), object_cols))
# 列名とユニーク値数の辞書を作成
d = dict(zip(object_cols, object_nunique))

# 列ごとのユニークな値の数を昇順で表示
# sorted関数でキーの値（ユニーク値の数）に基づいてソート
sorted(d.items(), key=lambda x: x[1])

[('Street', 2),
 ('Utilities', 2),
 ('CentralAir', 2),
 ('LandSlope', 3),
 ('PavedDrive', 3),
 ('LotShape', 4),
 ('LandContour', 4),
 ('ExterQual', 4),
 ('KitchenQual', 4),
 ('MSZoning', 5),
 ('LotConfig', 5),
 ('BldgType', 5),
 ('ExterCond', 5),
 ('HeatingQC', 5),
 ('Condition2', 6),
 ('RoofStyle', 6),
 ('Foundation', 6),
 ('Heating', 6),
 ('Functional', 6),
 ('SaleCondition', 6),
 ('RoofMatl', 7),
 ('HouseStyle', 8),
 ('Condition1', 9),
 ('SaleType', 9),
 ('Exterior1st', 15),
 ('Exterior2nd', 16),
 ('Neighborhood', 25)]

上記の出力は、カテゴリデータを持つ各列について、その列のユニークな値の数を示している。例えば、訓練データの`'Street'`列には、砂利道を表す`'Grvl'`と舗装道路を表す`'Pave'`という2つのユニークな値がある。

カテゴリ変数のユニークな値の数を、そのカテゴリ変数の**カーディナリティ**と呼ぶ。例えば、`'Street'`変数のカーディナリティは2である。

上記の出力を使用して、以下の質問に答える。

In [16]:
# 訓練データ内でカーディナリティが10より大きいカテゴリ変数はいくつありますか？
high_cardinality_numcols = 3

# 訓練データ内の'Neighborhood'変数をワンホットエンコードするには何列必要ですか？
num_cols_neighborhood = 25

# Kaggle環境での回答確認用（コメントアウト）
# step_3.a.check()

<IPython.core.display.Javascript object>

<span style="color:#33cc33">Correct</span>

In [17]:
# ヒントや解答コードを表示するための行（Kaggle環境用）
#step_3.a.hint()
#step_3.a.solution()

多くの行を持つ大規模なデータセットでは、ワンホットエンコーディングによってデータセットのサイズが大幅に拡大する可能性がある。このため、通常はカーディナリティが比較的低い列のみをワンホットエンコードする。そして、カーディナリティが高い列はデータセットから削除するか、ラベルエンコーディングを使用する。

例として、10,000行のデータセットで、100のユニークなエントリを持つ1つのカテゴリ列を考える。
- この列を対応するワンホットエンコーディングに置き換えると、データセットにいくつのエントリが追加されるか？
- 代わりに列をラベルエンコーディングに置き換えると、いくつのエントリが追加されるか？

回答を使用して、以下の行を埋める。

In [18]:
# 列をワンホットエンコーディングに置き換えることでデータセットに追加されるエントリ数は？
# 10,000行 × (100-1)列 = 990,000エントリ
# 元の列を削除して100列追加するので、実質99列の増加
OH_entries_added = 10000 * 99

# 列をラベルエンコーディングに置き換えることでデータセットに追加されるエントリ数は？
# ラベルエンコーディングは元の列を整数に置き換えるだけなので、追加エントリはない
label_entries_added = 0

# Kaggle環境での回答確認用（コメントアウト）
# step_3.b.check()

<IPython.core.display.Javascript object>

<span style="color:#33cc33">Correct</span>

In [19]:
# ヒントや解答コードを表示するための行（Kaggle環境用）
#step_3.b.hint()
#step_3.b.solution()

# ステップ4: ワンホットエンコーディング

このステップでは、ワンホットエンコーディングを試す。ただし、データセット内のすべてのカテゴリ変数をエンコードするのではなく、カーディナリティが10未満の列に対してのみワンホットエンコーディングを作成する。

以下のコードセルを変更せずに実行して、ワンホットエンコードされる列を含むPythonリスト`low_cardinality_cols`を設定する。同様に、`high_cardinality_cols`にはデータセットから削除されるカテゴリ列のリストが含まれる。

In [20]:
# ワンホットエンコードされる列
# カーディナリティが10未満の列を選択
low_cardinality_cols = [col for col in object_cols if X_train[col].nunique() < 10]

# データセットから削除される列
# 集合の差集合演算で、すべてのカテゴリ列から低カーディナリティ列を引く
high_cardinality_cols = list(set(object_cols)-set(low_cardinality_cols))

print('Categorical columns that will be one-hot encoded:', low_cardinality_cols)
print('\nCategorical columns that will be dropped from the dataset:', high_cardinality_cols)

Categorical columns that will be one-hot encoded: ['MSZoning', 'Street', 'LotShape', 'LandContour', 'Utilities', 'LotConfig', 'LandSlope', 'Condition1', 'Condition2', 'BldgType', 'HouseStyle', 'RoofStyle', 'RoofMatl', 'ExterQual', 'ExterCond', 'Foundation', 'Heating', 'HeatingQC', 'CentralAir', 'KitchenQual', 'Functional', 'PavedDrive', 'SaleType', 'SaleCondition']

Categorical columns that will be dropped from the dataset: ['Neighborhood', 'Exterior2nd', 'Exterior1st']


次のコードセルを使用して、`X_train`と`X_valid`のデータをワンホットエンコードする。前処理されたDataFrameをそれぞれ`OH_X_train`と`OH_X_valid`に設定する。

- データセット内のカテゴリ列の完全なリストはPythonリスト`object_cols`にある。
- `low_cardinality_cols`のカテゴリ列のみをワンホットエンコードする必要がある。他のすべてのカテゴリ列はデータセットから削除する必要がある。

In [21]:
from sklearn.preprocessing import OneHotEncoder

# OneHotEncoderを初期化
# handle_unknown='ignore': 未知のカテゴリに対してエラーを発生させない
# sparse=False: 密な配列を返す（デフォルトはスパース行列）
OH_encoder = OneHotEncoder(handle_unknown='ignore', sparse=False)

# 低カーディナリティ列をワンホットエンコード
OH_cols_train = pd.DataFrame(OH_encoder.fit_transform(X_train[low_cardinality_cols]))
OH_cols_valid = pd.DataFrame(OH_encoder.transform(X_valid[low_cardinality_cols]))

# ワンホットエンコーディングによりインデックスが削除されたので、元に戻す
OH_cols_train.index = X_train.index
OH_cols_valid.index = X_valid.index

# カテゴリ列を削除（ワンホットエンコーディングに置き換える）
num_X_train = X_train.drop(object_cols, axis=1)
num_X_valid = X_valid.drop(object_cols, axis=1)

# 数値特徴量にワンホットエンコードされた列を追加
OH_X_train = pd.concat([num_X_train, OH_cols_train], axis=1)
OH_X_valid = pd.concat([num_X_valid, OH_cols_valid], axis=1)

# Kaggle環境での回答確認用（コメントアウト）
# step_4.check()

<IPython.core.display.Javascript object>

<span style="color:#33cc33">Correct</span>

In [22]:
# ヒントや解答コードを表示するための行（Kaggle環境用）
#step_4.hint()
#step_4.solution()

このアプローチのMAEを取得するために次のコードセルを実行する。

In [23]:
print("MAE from Approach 3 (One-Hot Encoding):") 
print(score_dataset(OH_X_train, OH_X_valid, y_train, y_valid))

MAE from Approach 3 (One-Hot Encoding):
17525.345719178084


# ステップ5: テスト予測を生成し、結果を提出する

ステップ4を完了した後、学んだことを使用してリーダーボードに結果を提出したい場合は、予測を生成する前にテストデータを前処理する必要がある。

**このステップは完全にオプションであり、エクササイズを正常に完了するためにリーダーボードに結果を提出する必要はない。**

[コンペティションに参加する](https://www.kaggle.com/c/home-data-for-ml-course)方法や結果をCSVに保存する方法を思い出すのに助けが必要な場合は、前のエクササイズを確認してください。結果ファイルを生成したら、以下の手順に従ってください：
- 右上隅の青い**COMMIT**ボタンをクリックする。これによりポップアップウィンドウが生成される。
- コードの実行が終了したら、ポップアップウィンドウの右上にある青い**Open Version**ボタンをクリックする。これにより、同じページのビューモードに移動する。これらの指示に戻るには下にスクロールする必要がある。
- 画面左側の**Output**タブをクリックする。次に、**Submit to Competition**ボタンをクリックして、結果をリーダーボードに提出する。
- パフォーマンスを向上させるためにさらに作業を続けたい場合は、画面右上の青い**Edit**ボタンを選択する。その後、モデルを変更してプロセスを繰り返すことができる。

In [24]:
# テストデータの欠損値を0で埋める
X_test.fillna(0, inplace=True)

# すべてのカテゴリ列
object_cols = [col for col in X.columns if X[col].dtype == "object"]

# 安全にラベルエンコードできる列
# テストデータの値が訓練データの値のサブセットであることを確認
good_label_cols = [col for col in object_cols if 
                   set(X_test[col]).issubset(set(X[col]))]
        
# データセットから削除される問題のある列
bad_label_cols = list(set(object_cols)-set(good_label_cols))
        
print('Categorical columns that will be label encoded:', good_label_cols)
print('\nCategorical columns that will be dropped from the dataset:', bad_label_cols)

# エンコードされないカテゴリ列を削除
label_X = X.drop(bad_label_cols, axis=1)
label_X_test = X_test.drop(bad_label_cols, axis=1)

# ラベルエンコーダーを適用
labelEncoder = LabelEncoder()
for col in good_label_cols:
    # 全訓練データでエンコーダーをフィットし、変換
    label_X[col] = labelEncoder.fit_transform(X[col])
    # テストデータを変換
    label_X_test[col] = labelEncoder.transform(X_test[col])

# 200の決定木を持つモデル、MAE: 15923.57616
model = RandomForestRegressor(n_estimators=200, random_state=0)
model.fit(label_X, y)

# テスト予測を取得
preds_test = model.predict(label_X_test)

# テスト予測をファイルに保存
output = pd.DataFrame({'Id': label_X_test.index,
                       'SalePrice': preds_test})
output.to_csv('submission.csv', index=False)

Categorical columns that will be label encoded: ['Street', 'LotShape', 'LandContour', 'LotConfig', 'LandSlope', 'Neighborhood', 'Condition1', 'Condition2', 'BldgType', 'HouseStyle', 'RoofStyle', 'RoofMatl', 'ExterQual', 'ExterCond', 'Foundation', 'Heating', 'HeatingQC', 'CentralAir', 'PavedDrive', 'SaleCondition']

Categorical columns that will be dropped from the dataset: ['MSZoning', 'KitchenQual', 'Functional', 'Exterior2nd', 'Utilities', 'SaleType', 'Exterior1st']


# 次のステップ

欠損値の処理とカテゴリエンコーディングにより、モデリングプロセスは複雑になっている。この複雑さは、将来使用するためにモデルを保存したい場合にさらに悪化する。この複雑さを管理するための鍵は**パイプライン**と呼ばれるものだ。

**[パイプラインの使用方法を学ぶ](https://www.kaggle.com/alexisbcook/pipelines)**で、カテゴリ変数、欠損値、およびデータが投げかけるその他の厄介な問題を含むデータセットを前処理する方法を学ぼう。

---
**[中級機械学習ホームページ](https://www.kaggle.com/learn/intermediate-machine-learning)**





*質問やコメントがありますか？[Learn Discussion forum](https://www.kaggle.com/learn-forum)で他の学習者とチャットしましょう。*